In [1]:

import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
from sklearn.model_selection import train_test_split
import os
import  random
from tensorflow.keras import layers
import keras as keras
from keras.applications import  Xception
from PIL import Image
import random
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)
np.set_printoptions(precision=3, suppress=True)

import pandas as pd #library for data analysis
import matplotlib.pyplot as plt #library for creating static, animated, and interactive visualizations
import cv2 #Manipulate images and videos,
from PIL import Image # Python Imaging Library adds image processing capabilities to your Python interpreter. 
import math #mathematical functions
import requests #library for making HTTP requests

import faiss



2025-11-06 15:29:53.384418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762439393.407913    1897 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762439393.415117    1897 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762439393.435793    1897 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762439393.435832    1897 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762439393.435835    1897 computation_placer.cc:177] computation placer alr

2.19.0
Num GPUs Available:  1


In [6]:
model = tf.keras.models.load_model('../checkpoints/model_checkpoint_02.keras')


2025-11-06 15:34:29.095490: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-11-06 15:34:30.047850: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.


In [ ]:


embedding_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer('embedding').output
)
from io import BytesIO

WINDOWS_DATA_FOLDER="/mnt/e/code/ai/data/tcg/data/tcg/"


embeddings_db = {}
card_ids = []

for json_key in os.listdir(WINDOWS_DATA_FOLDER):
    data = json.load(open( f"{WINDOWS_DATA_FOLDER}{json_key}", "r"))
    if data["id"].startswith("base1"):
        response = requests.get(data["images"]["small"],headers={ 'Accept': 'image/jpg'})
        img = Image.open(BytesIO(response.content))
        if img is not None:
            img = img.resize((224, 224))
            img = np.expand_dims(img, axis=0)  # Ajouter une dimension batch
            embedding = embedding_model.predict(img, verbose=0)[0]
            # Normaliser l'embedding
            embedding = embedding / np.linalg.norm(embedding)
        
            embeddings_db[data["id"]] = embedding
            card_ids.append(data["id"])

            print(f"Embedding: {embedding}")
        
np.save("embeddings_db.npy", embeddings_db)
np.save("card_ids.npy", card_ids)



I0000 00:00:1762439743.900460    1944 service.cc:152] XLA service 0x70e9e8003fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762439743.905078    1944 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2025-11-06 15:35:44.287639: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1762439745.114303    1944 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1762439750.383284    1944 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Embedding: [0.003 0.    0.    0.    0.    0.    0.    0.025 0.    0.084 0.098 0.009
 0.081 0.    0.051 0.09  0.145 0.    0.077 0.061 0.035 0.    0.067 0.
 0.    0.    0.097 0.    0.19  0.163 0.    0.    0.02  0.    0.135 0.041
 0.161 0.    0.059 0.    0.035 0.    0.    0.    0.    0.038 0.    0.
 0.001 0.166 0.    0.046 0.    0.    0.    0.07  0.    0.048 0.    0.019
 0.    0.    0.    0.    0.    0.001 0.002 0.087 0.061 0.04  0.    0.
 0.249 0.056 0.    0.    0.    0.055 0.128 0.    0.    0.    0.014 0.098
 0.    0.002 0.    0.11  0.    0.023 0.    0.087 0.087 0.023 0.148 0.064
 0.005 0.018 0.012 0.039 0.    0.183 0.161 0.    0.    0.028 0.092 0.
 0.073 0.105 0.    0.    0.16  0.    0.    0.07  0.    0.168 0.074 0.067
 0.02  0.    0.    0.031 0.099 0.    0.117 0.    0.    0.071 0.079 0.178
 0.112 0.    0.031 0.083 0.128 0.    0.197 0.044 0.083 0.175 0.    0.
 0.    0.051 0.    0.    0.079 0.    0.    0.128 0.    0.048 0.141 0.
 0.    0.    0.    0.123 0.    0.    0.    0.073 0.    0.0

In [15]:

# Charger les embeddings et les card_ids
embeddings_db = np.load("embeddings_db.npy", allow_pickle=True).item()
card_ids = np.load("card_ids.npy", allow_pickle=True)

# Convertir les embeddings en tableau numpy
embeddings = np.array(list(embeddings_db.values()))

# Créer un index FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Utilise le produit scalaire (équivalent à la similarité cosinus après normalisation)
index.add(embeddings)

# Sauvegarder l'index FAISS
faiss.write_index(index, "embeddings_index.faiss")

def find_similar_cards(query_img, index, card_ids, embedding_model, k=3):
    # Extraire l'embedding de la nouvelle carte
    query_img = np.expand_dims(query_img, axis=0)
    query_embedding = embedding_model.predict(query_img, verbose=0)[0]
    # Normaliser l'embedding
    query_embedding = query_embedding / np.linalg.norm(query_embedding)

    # Rechercher les k plus proches voisins
    distances, indices = index.search(np.expand_dims(query_embedding, axis=0), k)

    # Récupérer les card_ids correspondants
    similar_cards = [(card_ids[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_cards



# Exemple d'utilisation
query_img = Image.open("base1-7_1762430565.jpg")
similar_cards = find_similar_cards(query_img, index, card_ids, embedding_model, k=3)
print("Cartes similaires :", similar_cards)

Cartes similaires : [(np.str_('base1-65'), np.float32(0.8643346)), (np.str_('base1-102'), np.float32(0.8412103)), (np.str_('base1-101'), np.float32(0.8359425))]
